In [1]:
# Load main packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import getpass # For identifying user/directory structure
import networkx as nx
import re
from sklearn import linear_model
%matplotlib inline

# Load local files
import sys # For reading files in other directories
if(getpass.getuser() == 'rockc_000'):
    sys.path.insert(0, 'C:/Users/rockc_000/Documents/GitHub/GithubSandbox/Python/NCAA Scripts')
    dir = 'C:/Users/rockc_000/Documents/Personal Files/Kaggle/NCAA'
if(getpass.getuser() == 'josh'):
    sys.path.insert(0, '/home/josh/Documents/Github/GithubSandbox/Python/NCAA Scripts')
    dir = '/home/josh/Documents/Personal/Kaggle/NCAA'
from make_submission import make_submission

In [2]:
# Read in data
seasons = pd.read_csv(dir + '/Seasons.csv')
teams = pd.read_csv(dir + '/Teams.csv')
regular_compact_results = pd.read_csv(dir + '/RegularSeasonCompactResults.csv')
regular_detailed_results = pd.read_csv(dir + '/RegularSeasonDetailedResults.csv')
tourney_compact_results = pd.read_csv(dir + '/TourneyCompactResults.csv')
tourney_detailed_results = pd.read_csv(dir + '/TourneyDetailedResults.csv')
tourney_seed = pd.read_csv(dir + '/TourneySeeds.csv')
tourney_slots = pd.read_csv(dir + '/TourneySlots.csv')
sample_submission = pd.read_csv(dir + '/SampleSubmission.csv')

In [3]:
# Look at the first few rows of each dataset, and their dimensions
# sample_submission.iloc[np.arange(5)]
nrow = 4
print('seasons:\n', seasons.head(nrow), '\nDimension: ', seasons.shape)
print('teams:\n', teams.head(nrow), '\nDimension: ', teams.shape)
print('regular_compact_results:\n', regular_compact_results.head(nrow),
      '\nDimension: ', regular_compact_results.shape)
print('regular_detailed_results:\n', regular_detailed_results.head(nrow),
      '\nDimension: ', regular_detailed_results.shape)
print('tourney_compact_results:\n', tourney_compact_results.head(nrow),
      '\nDimension: ', tourney_compact_results.shape)
print('tourney_detailed_results:\n', tourney_detailed_results.head(nrow),
      '\nDimension: ', tourney_detailed_results.shape)
print('tourney_seed:\n', tourney_seed.head(nrow),
      '\nDimension: ', tourney_seed.shape)
print('tourney_slots:\n', tourney_slots.head(nrow),
      '\nDimension: ', tourney_slots.shape)
print('sample_submission:\n', sample_submission.head(nrow),
      '\nDimension: ', sample_submission.shape)

seasons:
    Season     Dayzero Regionw    Regionx    Regiony    Regionz
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988  11/02/1987    East    Midwest  Southeast       West 
Dimension:  (32, 6)
teams:
    Team_Id    Team_Name
0     1101  Abilene Chr
1     1102    Air Force
2     1103        Akron
3     1104      Alabama 
Dimension:  (364, 2)
regular_compact_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985      20   1228      81   1328      64    N      0
1    1985      25   1106      77   1354      70    H      0
2    1985      25   1112      63   1223      56    H      0
3    1985      25   1165      70   1432      54    H      0 
Dimension:  (139920, 8)
regular_detailed_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003      10   1104      68   1328      6

## Pagerank algorithm on historical wins

In [4]:
# Initialize DataFrame with one (meaningless) row
pagerank_score = pd.DataFrame({'Season': [1900], 'Team': [0], 'Score': [0]})
for season in seasons['Season']:
    G = nx.DiGraph()
    edges = regular_compact_results.ix[regular_compact_results['Season'] == season,
                                       ('Wteam', 'Lteam')]
    for row in np.arange(edges.shape[0]):
        G.add_edge(edges.iloc[row][1], edges.iloc[row][0])
    # Calculate the page-rank using the networkx package
    out = nx.pagerank(G)
    # Convert the page-rank scores into a DataFrame.
    out = pd.DataFrame.from_dict(out, orient='index')
    out.rename(columns={0: 'Score'}, inplace=True)
    out['Team'] = out.index
    out['Season'] = season
    pagerank_score = pagerank_score.append(out)
pagerank_score = pagerank_score.ix[pagerank_score['Team'] != 0]

## Aggregated Statistics

In [10]:
win_count = regular_compact_results.groupby(('Season', 'Wteam'))
win_count = win_count['Daynum'].count()

# Aggregate the scores for the winning team and losing team
# as well as the number of games played.
win_pts = regular_compact_results.groupby(('Season', 'Wteam'))
win_pts = pd.merge(win_pts[('Lscore', 'Wscore')].sum(),
                    pd.DataFrame(win_pts['Season'].count()),
                    left_index=True, right_index=True)
lose_pts = regular_compact_results.groupby(('Season', 'Lteam'))
lose_pts = pd.merge(lose_pts[('Lscore', 'Wscore')].sum(),
                    pd.DataFrame(lose_pts['Season'].count()),
                    left_index=True, right_index=True)

win_pts.index.set_names(('Season', 'Team'), inplace=True)
lose_pts.index.set_names(('Season', 'Team'), inplace=True)
lose_pts.rename(columns={'Lscore': 'Points_for', 'Wscore': 'Points_againt',
                         'Season': 'Game_cnt'}, inplace=True)
win_pts.rename(columns={'Wscore': 'Points_for', 'Lscore': 'Points_againt',
                        'Season': 'Game_cnt'}, inplace=True)
total_pts = win_pts.append(lose_pts)
total_pts = total_pts.groupby(level=('Season', 'Team')).sum()

In [13]:
print(total_pts.loc[1985].loc[1102])
print(win_pts.loc[1985].loc[1102])
print(lose_pts.loc[1985].loc[1102])

Game_cnt           24
Points_againt    1653
Points_for       1514
Name: 1102, dtype: int64
Points_againt    305
Points_for       355
Game_cnt           5
Name: 1102, dtype: int64
Points_for       1159
Points_againt    1348
Game_cnt           19
Name: 1102, dtype: int64


In [50]:
#stat_vars = np.array(('or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf'),
#                     dtype=str)
#win_vars = np.array(['W' + x for x in stat_vars])
#lose_vars = np.array(['L' + x for x in stat_vars])

# Add a column to make counting games easier:
regular_detailed_results['all_one'] = 1
detail_win = regular_detailed_results.groupby(('Season', 'Wteam'))
detail_win = detail_win[('Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf',
                         'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf',
                         'all_one')].sum()
def col_rename(col_name):
    if(re.search('W', col_name) is not None):
        col_name = re.sub('W', '', col_name)
        col_name = col_name + '_for'
    if(re.search('L', col_name) is not None):
        col_name = re.sub('L', '', col_name)
        col_name = col_name + '_against'
    return(col_name)
detail_win.rename(columns=col_rename, inplace=True)
detail_win.index.set_names(('Season', 'Team'), inplace=True)

detail_lose = regular_detailed_results.groupby(('Season', 'Lteam'))
detail_lose = detail_lose[('Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf',
                           'Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf',
                           'all_one')].sum()
def col_rename(col_name):
    if(re.search('W', col_name) is not None):
        col_name = re.sub('W', '', col_name)
        col_name = col_name + '_against'
    if(re.search('L', col_name) is not None):
        col_name = re.sub('L', '', col_name)
        col_name = col_name + '_for'
    return(col_name)
detail_lose.rename(columns=col_rename, inplace=True)
detail_lose.index.set_names(('Season', 'Team'), inplace=True)

detail_data = detail_win.append(detail_lose)
detail_data = detail_data.groupby(level=('Season', 'Team')).sum()
detail_data.drop(labels='all_one', axis=1, inplace=True)

## Other variables

- It'd be interesting to look at trends.  So, we could fit a linear regression to the point difference over time, and see if they tend to be winning or losing by more and more points.

## Create final DataFrame

In [51]:
print(detail_data.shape)
print(total_pts.shape)
print(win_count.shape)
final_data = pd.merge(detail_data, total_pts, right_index=True, left_index=True, how='outer')
win_count = pd.DataFrame(win_count)
win_count.index.rename(('Season', 'Team'), inplace=True)
final_data = pd.merge(final_data, win_count, right_index=True, left_index=True, how='outer')
final_data

(4428, 14)
(9835, 3)
(9821, 1)


or_for  dr_for  ast_for  to_for  stl_for  blk_for  pf_for  \
Season Team                                                              
1985   1102     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1103     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1104     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1106     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1108     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1109     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1110     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1111     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1112     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1113     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1114     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1116     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1117     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1119     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1120     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1121     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1122     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1123     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1124     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1126     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1129     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1130     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1131     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1132     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1133     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1134     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1135     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1137     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1139     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
       1140     NaN     NaN      NaN     NaN      NaN      NaN     NaN   
...             ...     ...      ...     ...      ...      ...     ...   
2015   1433     372     785      442     364      338      151     673   
       1434     284     699      371     355      169      130     498   
       1435     256     653      472     385      151      149     533   
       1436     210     669      385     377      198      156     578   
       1437     307     770      541     365      265      137     562   
       1438     295     766      413     298      179      137     456   
       1439     225     560      396     382      144       97     595   
       1440     326     630      357     422      217      157     609   
       1441     320     664      349     401      238      131     637   
       1442     152     526      300     316      130       60     451   
       1443     283     730      410     387      155       96     544   
       1444     250     660      380     361      197       66     579   
       1447     364     675      302     396      181       75     661   
       1448     321     803      451     425      208      123     579   
       1449     271     672      412     357      152      153     534   
       1450     233     673      453     358      120      102     570   
       1451     230     610      316     367      140      107     509   
       1452     462     648      465     412      350       91     748   
       1453     297     688      415     352      225      142     553   
       1454     198     595      366     361      150       79     488   
       1455     307     688      423     278      219      117     514   
       1456     1

## Quick Correlation Plots